In [2]:
import xarray as xr
import numpy as np
import datetime

# Unification of daily data

In [4]:
path = "../data/SCS/"

chl_path = path + "MODIS_CHL_DAILY/"
temp_path = path + "MODIS_TEMP_DAILY/"

data_path = path + "MODIS_INTERPOLATED_DATA/"

In [103]:
chl = xr.open_mfdataset(chl_path + "AQUA_MODIS.*.L3m.DAY.CHL.x_chlor_a.nc", combine = "nested", concat_dim="time")

# BASH COMMAND TO USE FOR dates.txt CREATION
# printf '%s\n' * | cut -d . -f2 >> dates.txt

with open(chl_path + "dates.txt", 'rb') as file:
     data = file.read()

dates_list = data.decode("utf-16").split("\r\n")

chl["time"] = [datetime.datetime.strptime(i, "%Y%m%d") for i in dates_list]
chl.chlor_a.load().to_netcdf(data_path + "MODIS_chl_extended.nc")

In [104]:
temp = xr.open_mfdataset(temp_path + "AQUA_MODIS.*.L3m.DAY.SST.x_sst.nc", combine = "nested", concat_dim="time")

# BASH COMMAND TO USE FOR dates.txt CREATION
# printf '%s\n' * | cut -d . -f2 >> dates.txt

with open(temp_path + "/dates.txt", 'rb') as file:
     data = file.read()

dates_list = data.decode("utf-16").split("\r\n")

temp["time"] = [datetime.datetime.strptime(i, "%Y%m%d") for i in dates_list]

temp.sst.load().to_netcdf(data_path + "MODIS_temp_extended.nc")

In [99]:
lons, lats = temp.lon, temp.lat

bathy = xr.open_dataset(path + "OLD_DATA/bathymetry.nc")

bathy = bathy.interp(lon = lons, lat = lats, method = "nearest")

bathy.to_netcdf(data_path + "bathymetry_interpolated.nc")

# 8-day average

In [105]:
chlorophyll = xr.open_dataset(data_path + "MODIS_chl_extended.nc").load().convert_calendar("noleap")

chlorophyll = chlorophyll.sel(time=~((chlorophyll.time.dt.month == 12) & (chlorophyll.time.dt.day >= 27))).resample(time = "1D").asfreq()

c = chlorophyll.where(chlorophyll["time.year"] == 2003, drop = True).resample(time = "8D").mean()
for y in range(2004,2022):
    c = xr.merge([c, chlorophyll.where(chlorophyll["time.year"] == y, drop = True).resample(time = "8D").mean()])

datetimeindex = c.indexes['time'].to_datetimeindex()

c["time"] = datetimeindex

c.load().to_netcdf(data_path + "MODIS_chl_extended_8D.nc")

c:\Users\feder\miniconda3\lib\site-packages\xarray\coding\times.py:351: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  sample = dates.ravel()[0]
C:\Users\feder\AppData\Local\Temp\ipykernel_10548\2447480424.py:9: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  datetimeindex = c.indexes['time'].to_datetimeindex()


In [106]:
temperature = xr.open_dataset(data_path + "MODIS_temp_extended.nc").load().convert_calendar("noleap")

temperature = temperature.sel(time=~((temperature.time.dt.month == 12) & (temperature.time.dt.day >= 27))).resample(time = "1D").asfreq()

c = temperature.where(temperature["time.year"] == 2003, drop = True).resample(time = "8D").mean()
for y in range(2004,2022):
    c = xr.merge([c, temperature.where(temperature["time.year"] == y, drop = True).resample(time = "8D").mean()])

datetimeindex = c.indexes['time'].to_datetimeindex()

c["time"] = datetimeindex

c.load().to_netcdf(data_path + "MODIS_temp_extended_8D.nc")

c:\Users\feder\miniconda3\lib\site-packages\xarray\coding\times.py:351: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  sample = dates.ravel()[0]
C:\Users\feder\AppData\Local\Temp\ipykernel_10548\4051978394.py:9: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  datetimeindex = c.indexes['time'].to_datetimeindex()


# Computation of anomalies

In [5]:
chlorophyll = xr.open_dataset(data_path + "MODIS_chl_extended_8D.nc")
temperature = xr.open_dataset(data_path + "MODIS_temp_extended_8D.nc")

chlorophyll = chlorophyll.assign_coords(day_of_year = chlorophyll.time.dt.strftime("%d-%m"))
chl_anomalies = ((chlorophyll.groupby("day_of_year") - chlorophyll.groupby("day_of_year").mean("time")).groupby("day_of_year")) / chlorophyll.groupby("day_of_year").std("time")

temperature = temperature.assign_coords(day_of_year = temperature.time.dt.strftime("%d-%m"))
temp_anomalies = ((temperature.groupby("day_of_year") - temperature.groupby("day_of_year").mean("time")).groupby("day_of_year")) / temperature.groupby("day_of_year").std("time")

c:\Users\feder\miniconda3\lib\site-packages\numpy\lib\nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\feder\miniconda3\lib\site-packages\numpy\lib\nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\feder\miniconda3\lib\site-packages\numpy\lib\nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\feder\miniconda3\lib\site-packages\numpy\lib\nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\feder\miniconda3\lib\site-packages\numpy\lib\nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\feder\miniconda3\lib\site-packages\numpy\lib\n

In [7]:
chl_anomalies.load().to_netcdf(data_path + "MODIS_chl_anomalies_8D.nc")
temp_anomalies.load().to_netcdf(data_path + "MODIS_temp_anomalies_8D.nc")